In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

import time

In [61]:
base = np.arange(7*7).reshape(7,7)

mask = np.zeros((3,4), dtype=int)
#mask[1,0]=1
mask[1,-1]=1
#mask[-1,-1]=1

#mask[0,1]=10
#mask[1,0]=100
#mask[1,1]=1000
#mask[2,0]=10000
#mask[2,1]=100000
print(base)
print(mask)

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]]
[[0 0 0 0]
 [0 0 0 1]
 [0 0 0 0]]


In [62]:
#signal.convolve2d(base, mask, boundary='wrap', mode='same').astype(int)

flip_mask = mask[::-1,::-1]
print(flip_mask.astype(int))
print()
res = signal.convolve2d(base, flip_mask, boundary='wrap', mode='full').astype(int)
print(res.shape)
print(res)
print()
res = signal.convolve2d(base, flip_mask, boundary='wrap', mode='same').astype(int)
print(res.shape)
print(res)
print()
res = signal.convolve2d(base, flip_mask, boundary='wrap', mode='valid').astype(int)
print(res)

[[0 0 0 0]
 [1 0 0 0]
 [0 0 0 0]]

(9, 10)
[[42 43 44 45 46 47 48 42 43 44]
 [ 0  1  2  3  4  5  6  0  1  2]
 [ 7  8  9 10 11 12 13  7  8  9]
 [14 15 16 17 18 19 20 14 15 16]
 [21 22 23 24 25 26 27 21 22 23]
 [28 29 30 31 32 33 34 28 29 30]
 [35 36 37 38 39 40 41 35 36 37]
 [42 43 44 45 46 47 48 42 43 44]
 [ 0  1  2  3  4  5  6  0  1  2]]

(7, 7)
[[ 1  2  3  4  5  6  0]
 [ 8  9 10 11 12 13  7]
 [15 16 17 18 19 20 14]
 [22 23 24 25 26 27 21]
 [29 30 31 32 33 34 28]
 [36 37 38 39 40 41 35]
 [43 44 45 46 47 48 42]]

[[10 11 12 13]
 [17 18 19 20]
 [24 25 26 27]
 [31 32 33 34]
 [38 39 40 41]]


In [28]:
signal.convolve2d(base, mask, boundary='wrap',mode='same')

array([[48, 42, 43, 44, 45, 46, 47],
       [ 6,  0,  1,  2,  3,  4,  5],
       [13,  7,  8,  9, 10, 11, 12],
       [20, 14, 15, 16, 17, 18, 19],
       [27, 21, 22, 23, 24, 25, 26],
       [34, 28, 29, 30, 31, 32, 33],
       [41, 35, 36, 37, 38, 39, 40]])

In [29]:
np.arange(16).reshape(4,4)[::-1,::-1]

array([[15, 14, 13, 12],
       [11, 10,  9,  8],
       [ 7,  6,  5,  4],
       [ 3,  2,  1,  0]])

In [54]:
image = np.random.random((200,200))
mask6 = np.random.random((6,6))
mask12 = np.random.random((12,12))
fmask = mask[::-1,::-1]

%timeit signal.convolve2d(image, mask6, boundary='wrap', mode='same')
%timeit signal.convolve2d(image, mask12, boundary='wrap', mode='same')

6.49 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
22.9 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
cv = signal.convolve2d(image, mask, boundary='wrap', mode='same')
cr = signal.correlate2d(image, fmask, boundary='wrap', mode='same')

In [48]:
cv == cr

array([[False, False, False, ...,  True,  True, False],
       [False,  True,  True, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True, False, ..., False,  True,  True],
       [ True, False, False, ..., False, False, False],
       [ True,  True, False, ..., False, False,  True]])

In [53]:
np.allclose(cr, cv)

True

In [50]:
cv

array([[5.14463515, 6.04120921, 5.29782864, ..., 5.63586648, 5.13108276,
        5.65025701],
       [5.67155675, 6.30800782, 6.38559348, ..., 5.92333274, 5.66826246,
        6.21848205],
       [5.67356153, 6.41552537, 4.95321534, ..., 6.09018424, 6.72826436,
        5.79898199],
       ...,
       [5.28560711, 5.14848683, 5.01442728, ..., 4.41575517, 4.36236423,
        4.79092676],
       [4.24056796, 5.50804804, 4.9941046 , ..., 4.43119028, 4.76516484,
        4.73117204],
       [5.08706341, 5.75089236, 5.75311841, ..., 5.52993927, 5.36698028,
        4.68479507]])

In [75]:
np.arange(25).reshape(5,5)[::-1,::-1]

array([[24, 23, 22, 21, 20],
       [19, 18, 17, 16, 15],
       [14, 13, 12, 11, 10],
       [ 9,  8,  7,  6,  5],
       [ 4,  3,  2,  1,  0]])